### Load packages and arrays

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
image_array = np.load('/Users/Danielle/Documents/Arrays/image_array.npy')
label_array = np.load('/Users/Danielle/Documents/Arrays/pathology_array.npy')

### Split into training (75%) and test (25%) datasets

In [3]:
#split using train_test_split function
x_train, x_test, y_train, y_test = train_test_split(image_array, label_array, test_size=0.25)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(18572, 32, 32) (18572, 1)
(6191, 32, 32) (6191, 1)


In [4]:
#change and define input shape
x_train = x_train.reshape(x_train.shape[0], 32, 32,1)
x_test = x_test.reshape(x_test.shape[0], 32, 32,1)
input_shape = (32, 32, 1)

### Create model

In [5]:
#access virtual environment with tensorflow
os.environ["/Users/Danielle/anaconda3/envs/tensorflow"] = "1"

In [6]:
#load tensorflow packages/functions
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [7]:
#create model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=input_shape)) #input
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #output

In [8]:
#configure model for training
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adamax(),
              metrics=['accuracy'])

### Model training

In [9]:
#fit model
train_score = model.fit(x_train, y_train,
          epochs=10,
          verbose=1,
          validation_split=0.15) #ensure validation set created (15% of training set)

Train on 15786 samples, validate on 2786 samples
Epoch 1/10
15786/15786 [==============================] - 31s 2ms/step - loss: 0.6608 - acc: 0.5914 - val_loss: 0.5891 - val_acc: 0.6841
Epoch 2/10
15786/15786 [==============================] - 25s 2ms/step - loss: 0.4612 - acc: 0.7882 - val_loss: 0.4328 - val_acc: 0.8112
Epoch 3/10
15786/15786 [==============================] - 27s 2ms/step - loss: 0.2829 - acc: 0.8945 - val_loss: 0.3088 - val_acc: 0.8841
Epoch 4/10
15786/15786 [==============================] - 26s 2ms/step - loss: 0.1988 - acc: 0.9270 - val_loss: 0.3214 - val_acc: 0.8948
Epoch 5/10
15786/15786 [==============================] - 25s 2ms/step - loss: 0.1389 - acc: 0.9503 - val_loss: 0.2884 - val_acc: 0.8991
Epoch 6/10
15786/15786 [==============================] - 24s 2ms/step - loss: 0.1014 - acc: 0.9619 - val_loss: 0.3321 - val_acc: 0.8920
Epoch 7/10
15786/15786 [==============================] - 24s 2ms/step - loss: 0.0730 - acc: 0.9723 - val_loss: 0.3528 - val_acc:

### Model Results

In [10]:
#evaluate test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.46904170653957133
Test accuracy: 0.9014698756355362


In [11]:
#predict labels
predictions = model.predict(x_test)

#Save arrays for plotting later
np.save("/Users/Danielle/Documents/Arrays/predicted", predictions)
np.save("/Users/Danielle/Documents/Arrays/test_labels", y_test)

In [12]:
#confusion matrix
predictions = np.where(predictions>0.5, 1, 0) #threshold predicted values > 0.5 <
c = confusion_matrix(y_test, predictions)
print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))
print('confusion matrix', c)

sensitivity 0.8981302385557705
specificity 0.904823567497572
confusion matrix [[2786  316]
 [ 294 2795]]


### Model Summary

In [13]:
print(model.summary()) #to use in model architecture diagram

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
__________